# Running database reconstruction attacks on the Iris dataset

In this tutorial we will show how to run a database reconstruction attack on the Iris dataset and evaluate its effectiveness against models trained non-privately (i.e., naively with scikit-learn) and models trained with differential privacy guarantees.

## Preliminaries

The database reconstruction attack takes a trained machine learning model `model`, which has been trained by a training dataset of `n` examples.  Then, using `n-1` examples of the training dataset (i.e., with the target row removed), we seek to reconstruct the `n`th example of the dataset by using `model`.

In this example, we train a Gaussian Naive Bayes classifier (`model`) with the training dataset, then remove a single row from that dataset, and seek to reconstruct that row using `model`. For typical examples, this attack is successful up to machine precision.

We then show that launching the same attack on a ML model trained with differential privacy guarantees provides protection for the training dataset, and prevents learning the target row with precision.

### Install PyEnv and Poetry

We use PyEnv to set a specific Python version declared in .python-version file, and Poetry to lock all Python direct and transient dependencies. We can log into Huggingface with a token stored in Colab secrets to get access to models that require accepting terms and conditions. We can mount Google Drive to store runs in a directory.

In [ ]:
# Install PyEnv
# !sudo apt update; sudo apt install build-essential libssl-dev zlib1g-dev  libbz2-dev libreadline-dev libsqlite3-dev curl git libncursesw5-dev xz-utils tk-dev libxml2-dev libxmlsec1-dev libffi-dev liblzma-dev
!rm -rf /root/.pyenv
!curl -fsSL https://pyenv.run | bash
import os
os.environ['PYENV_ROOT'] = os.environ['HOME'] + '/.pyenv'
pyenv_bin_dir = os.path.join(os.environ['HOME'], '.pyenv/bin')
os.environ['PATH'] = pyenv_bin_dir + ':' + os.environ['PATH']

# Install Poetry
!curl -sSL https://install.python-poetry.org | python3 -
import os
os.environ['PATH'] = '/root/.local/bin' + ':' + os.environ['PATH']
!which poetry


Cloning into '/root/.pyenv'...
remote: Enumerating objects: 1365, done.
remote: Counting objects: 100% (1365/1365), done.
remote: Compressing objects: 100% (727/727), done.
remote: Total 1365 (delta 827), reused 804 (delta 505), pack-reused 0 (from 0)
Receiving objects: 100% (1365/1365), 1.14 MiB | 2.52 MiB/s, done.
Resolving deltas: 100% (827/827), done.
Cloning into '/root/.pyenv/plugins/pyenv-doctor'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 1), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 38.72 KiB | 19.36 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Cloning into '/root/.pyenv/plugins/pyenv-update'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 1), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10),

In [ ]:
!git clone https://github.com/Trusted-AI/adversarial-robustness-toolbox.git
!pip install ./adversarial-robustness-toolbox

fatal: destination path 'adversarial-robustness-toolbox' already exists and is not an empty directory.
Processing ./adversarial-robustness-toolbox
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for adversarial-robustness-toolbox: filename=adversarial_robustness_toolbox-1.19.1-py3-none-any.whl size=1702166 sha256=1276f86b95d6173c8a4e61b6e0d290903001108bfd5e48662987d2f30584313f
  Stored in directory: /root/.cache/pip/wheels/74/56/11/32cf1e851d12b9ff6f8b14f8e747d8bf6474a4ee4ffe448e82
Successfully built adversarial-robustness-toolbox
  Attempting uninstall: adversarial-robustness-toolbox
    Found existing installation: adversarial-robustness-toolbox 1.19.1
    Uninstalling adversarial-robustness-toolbox-1.19.1:
      Successfully uninstalled adversarial-robustness-toolbox-1.19.1


## Example usage

## Load data

First, we load the data of interest and split into train/test subsets.

In [ ]:
!pip install scikit-learn diffprivlib

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

dataset = datasets.load_iris()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2)

## Train model

We can now train a Gaussian naive Bayes classifier using the full training dataset. This is the model that will be used to attack the training dataset later.

In [ ]:
import sklearn.naive_bayes as naive_bayes
from art.estimators.classification.scikitlearn import ScikitlearnGaussianNB

model1 = naive_bayes.GaussianNB().fit(x_train, y_train)
non_private_art = ScikitlearnGaussianNB(model1)

In [ ]:
print("Model accuracy (on the test dataset): {}".format(model1.score(x_test, y_test)))

Model accuracy (on the test dataset): 0.9666666666666667


## Launch and evaluate attack

We now select a row from the training dataset that we will remove. This is the **target row** which the attack will seek to reconstruct. The attacker will have access to `x_public` and `y_public`.

In [ ]:
target_row = int(np.random.random() * x_train.shape[0])

x_public = np.delete(x_train, target_row, axis=0)
y_public = np.delete(y_train, target_row, axis=0)

We can now launch the attack, and seek to infer the value of the target row. This is typically completed in less than a second.

In [ ]:
from art.attacks.inference.reconstruction import DatabaseReconstruction

dbrecon = DatabaseReconstruction(non_private_art)

x, y = dbrecon.reconstruct(x_public, y_public)

We can evaluate the accuracy of the attack using root-mean-square error (RMSE), showing a high level of accuracy in the inferred value.

In [ ]:
print("Inference RMSE: {}".format(
    np.sqrt(((x_train[target_row] - x) ** 2).sum() / x_train.shape[1])))

Inference RMSE: 4.787202022222637e-08


We can confirm that the attack also inferred the correct label `y`.

In [ ]:
np.argmax(y) == y_train[target_row]

np.True_

# Attacking a model trained with differential privacy

We can mitigate against this attack by training the public ML model with differential privacy.  We will use [diffprivlib](https://github.com/Trusted-AI/differential-privacy-library) to train a differentially private Gaussian naive Bayes classifier. We can mitigate against any loss in accuracy of the model by choosing an `epsilon` value appropriate to our needs.

## Train the model

In [ ]:
from diffprivlib import models

model2 = models.GaussianNB(bounds=([4.3, 2.0, 1.1, 0.1], [7.9, 4.4, 6.9, 2.5]), epsilon=3).fit(x_train, y_train)
private_art = ScikitlearnGaussianNB(model2)

model2.score(x_test, y_test)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


0.8333333333333334

## Launch and evaluate attack

We then launch the same attack as before. In this case, the attack may take a number of seconds to return a result.

In [ ]:
dbrecon = DatabaseReconstruction(private_art)

x_dp, y_dp = dbrecon.reconstruct(x_public, y_public)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator.

In this case, the RMSE shows our attack has not been as successful

In [ ]:
print("Inference RMSE (with differential privacy): {}".format(
    np.sqrt(((x_train[target_row] - x_dp) ** 2).sum() / x_train.shape[1])))

Inference RMSE (with differential privacy): 0.17320508075688767


This is confirmed by inspecting the inferred value and the true value.

In [ ]:
x_dp, x_train[target_row]

(array([[5.2, 3.4, 1.4, 0.2]]), array([5. , 3.2, 1.2, 0.2]))

In fact, the attack may not even be able to correctly infer the target label.

In [ ]:
np.argmax(y_dp), y_train[target_row]

(np.int64(2), np.int64(0))